In [46]:
import numpy as np
from copy import deepcopy
import pandas as pd
from numpy import log10, linspace, array, zeros, append, std, asarray, vstack, exp, zeros_like, arange, dot
import matplotlib.pyplot as plt
from math import log
import sympy as sp
from numpy.linalg import inv

In [16]:
data = pd.read_csv('../input/cdcsv/ClassificationDataset.csv')

In [24]:
datums_train = array(data['Class'][0:990])
datums_test = array(data['Class'][990:1000])
inputs_train = array(data.drop(columns=['Class']))[0:990, :]
inputs_test = array(data.drop(columns=['Class']))[990:1000, :].T

outputs_train = zeros(shape=(1, 990))
for i, d in enumerate(datums_train):
    if d == 'A':
        outputs_train[0, i] = 1
    else:
        outputs_train[0, i] = 0
    
outputs = vstack([outputs_train, abs(outputs_train[0]-1)])
inputs = inputs_train.T

In [25]:
class Neural_Network():
    def __init__(self, network_array): 
        
        no_of_layers = len(network_array)
        bias_array = zeros(shape=(1, int(no_of_layers - 1)))[0]
        
        self.no_of_layers = no_of_layers
        self.bias_array = bias_array
        self.network_array = network_array
        
        # make a blank net in dictionary form
        
        blank_net = {}
        for layer in arange(1,no_of_layers):
            no_of_nodes = network_array[layer]
            weights_per_node = network_array[int(layer-1)]
        
            Nodes_dict = {'B': bias_array[int(layer-1)]}
            for node in arange(no_of_nodes)+1:
                act = np.random.rand()
                Nodes_dict['N{0}'.format(node)] = {'A': act, 'Z': 0, 'W': [1]*weights_per_node}
  
            blank_net['L{0}'.format(layer)] = deepcopy(Nodes_dict)
            
        self.blank_net = blank_net
        
        
    # functions applied   
        
    def std(self, z):
        zs = zeros_like(z)
        for i, zi in enumerate(z.T[0]):
            if zi >= 100:
                zs[i, 0] = 1
            elif zi <= -100:
                zs[i, 0] = 0
            else:
                zs[i, 0] = 1/(1 + np.exp(-zi))
        return zs
    
    def dirr_std(self, z):
        zs = zeros_like(z)
        for i, zi in enumerate(z.T[0]):
            if zi >= 100:
                zs[i, 0] = 0
            elif zi <= -100:
                zs[i, 0] = 0
            else:
                zs[i, 0] = np.exp(-zi)/(1 + np.exp(-zi))**2
        return zs
    
    def output_func(self, y):
        for i, yi in enumerate(y.T[0]):
            if yi >= 100:
                y[i, 0] = 100
            elif yi <= -100:
                y[i, 0] = -100
        return np.exp(y)/(sum(np.exp(y)))
        
        
    # removes dictionary neural net properties in standard matrix forms    
        
    def dict_extract(self, layer_number, neural_net):
        NN = deepcopy(neural_net)
        Li = 'L{0}'.format(layer_number)
        no_of_nodes = self.network_array[layer_number]
        no_of_weights = self.network_array[int(layer_number-1)]
            
        B = NN[Li]['B']
        A = zeros(shape=(no_of_nodes, 1))
        Z = zeros(shape=(no_of_nodes, 1))
        W = zeros(shape=(no_of_nodes, no_of_weights))
        for node in arange(no_of_nodes)+1:
            A[int(node-1),0] = NN[Li]['N{0}'.format(node)]['A']
            Z[int(node-1),0] = NN[Li]['N{0}'.format(node)]['Z']
            W[int(node-1),:] = NN[Li]['N{0}'.format(node)]['W']
                
        return A, Z, W, B
    
    
    # returns matrix to neural net dictionary form
    
    def dict_compile(self, A, Z, W, B, layer_number, neural_net):
        NN = deepcopy(neural_net)
        Li = 'L{0}'.format(layer_number)
        no_of_nodes = self.network_array[layer_number]
        no_of_weights = self.network_array[int(layer_number-1)]
    
        NN[Li]['B'] = B
        for node in arange(no_of_nodes)+1:
            NN[Li]['N{0}'.format(node)]['A'] = A[int(node-1),0]
            NN[Li]['N{0}'.format(node)]['Z'] = Z[int(node-1),0]
            NN[Li]['N{0}'.format(node)]['W'] = list(W[int(node-1),:])
            
        return NN
    
    
    # returns the output of the neural net dictionary 
    
    def output(self, neural_net):
        NN = deepcopy(neural_net)
        no_of_nodes = self.network_array[-1]
        
        YA = zeros(shape=(no_of_nodes, 1))
        for node in arange(no_of_nodes)+1:
            YA[int(node-1), 0] = NN['L{0}'.format(int(self.no_of_layers-1))]['N{0}'.format(node)]['A']
        
        return YA
    
        
    def fore_prop(self, x, neural_net):
        A0 = x
            
        NN = deepcopy(neural_net)
            
        for layer in arange(1,self.no_of_layers):
            no_of_nodes = self.network_array[layer]
            weights_per_node = self.network_array[int(layer-1)]
            
            A1, Z1, W1, B1 = self.dict_extract(layer, NN)
            Z1 = dot(W1, A0) + B1
            A1 = self.std(Z1)
            A0 = A1
            
            NN = self.dict_compile(A1, Z1, W1, B1, layer, NN)
            
        return NN
    
    
    
    def back_prop(self, x, y, neural_net, alpha):
        NN = deepcopy(neural_net)
        
        dZ = self.output(NN) - y
        for layer in np.flip(arange(1,self.no_of_layers)):
            
            if layer == 1:
                A1, Z1, W1, B1 = self.dict_extract(layer, NN)
                dW = dot(dZ, x.T)
                dB = sum(dZ)

                W1 -= alpha*dW
                B1 -= alpha*dB

                NN = self.dict_compile(A1, Z1, W1, B1, layer, NN)
                
            else:
                A0, Z0 = self.dict_extract(int(layer-1), NN)[0:2]
                A1, Z1, W1, B1 = self.dict_extract(int(layer), NN)
                dW = dot(dZ, A0.T)
                dB = sum(dZ)
                dZ = dot(W1.T, dZ)*self.dirr_std(Z0)

                W1 -= alpha*dW
                B1 -= alpha*dB

                NN = self.dict_compile(A1, Z1, W1, B1, layer, NN)
            
        return NN
            
    
    
    def train(self, X, Y, trials, alpha):
        
        no_of_inputs, no_of_sets = X.shape
        NN = self.blank_net
        
        for trial in range(trials):
            for i in range(no_of_sets):
                x = array([X[:, i]]).T
                y = array([Y[:, i]]).T
                NN = self.fore_prop(x, NN)
                NN = self.back_prop(x, y, NN, alpha)
                
        return NN
    
    
    
    def predict(self, x, trained_net):
        NN = deepcopy(trained_net)
        NN = self.fore_prop(x, NN)
        
        return self.output(NN)
    

In [36]:
net = array([10, 2])
nn = Neural_Network(net)
tn = nn.train(inputs, outputs, 10, 0.1)

In [65]:
for entry in range(10): 

    test_input = array([inputs_test[:, entry]]).T
    test_output = array([datums_test[entry]])[0]

    ans = nn.predict(test_input, tn)
    if ans[0, 0] > ans[1, 0]:
        result = 'A'
    else:
        result = 'B'

    if result == test_output:
        print('correct!')
    else:
        print('wrong!')

In [45]:
tn